In [1]:
import pandas as pd
import numpy as np
#from sklearn.utils import shuffle

df = pd.read_csv('full_dataset.csv')
#df = shuffle(df, random_state=42)
df.shape

(974, 8)

In [2]:
df.head()
#kompletter Datensatz

,filename,corpus,genre,period,region,text,title,year
0,HUMA_P3_WMD_1777_HomburgRAW.txt,manchester,HUMA,P3,WMD,Nachricht von den Alterthu&#868;mern in dem Ge...,Homburg,1777
1,NEWS_P3_NoD_1786_wolfenbuettel1.txt,manchester,NEWS,P3,NoD,"Zeitung\r\nfür\r\nStädte, Flecken und Dörfer,\...",wolfenbuettel1,1786
2,NARR_P1_NoD_1658_MorgenlaendischRAW.txt,manchester,NARR,P1,NoD,Das zwey vnd dreysigste Capitel.\n Des Pr...,Morgenlaendisch,1658
3,SCIE_P1_WMD_1680_EpidemicaRAW.txt,manchester,SCIE,P1,WMD,Das XX. Capitel.\n Von den Schnecken.\n ...,Epidemica,1680
4,NEWS_P2_WMD_1701_hanau2.txt,manchester,NEWS,P2,WMD,Extraordinari Europæische Zeitung. 1701. Num. ...,hanau2,1701


# Achtung!!! Hier können die NEWS aus dem Datensatz herausgenommen werden - Einfach mit \# kommentieren oder entkommentieren!

In [3]:
#df = df[df.genre != 'NEWS']
#print(df)

# Teilung des Dataframes in die einzelnen Teilcorpora und Perioden

In [4]:
#Teilung in Manchester und Innsbruck Dataframe, sowie in die einzelnen Perioden
#Der einfachheit halber wird hier mit dem Dataframe.loc Attribut gearbeitet, weil es vergleichbar mit einem SQL-Statement ist ist
#Hierfür jeweils eine eigene Zelle, damit man via Shape nochmal die Größe prüfen kann
#Für die Tests können die einzelnen Dataframes wie gewünscht concateniert werden



df_man = df.loc[df['corpus'] == 'manchester']
df_man.shape

(336, 8)

In [5]:
df_inn = df.loc[df['corpus'] == 'innsbruck']
df_inn.shape 

(638, 8)

In [6]:
#P1
df_P1 = df.loc[df['period'] == 'P1']
df_P1.shape

(115, 8)

In [7]:
#P2
df_P2 = df.loc[df['period'] == 'P2']
df_P2.shape

(113, 8)

In [8]:
#P3
df_P3 = df.loc[df['period'] == 'P3']
df_P3.shape

(108, 8)

In [9]:
#P4
df_P4 = df.loc[df['period'] == 'P4']
df_P4.shape

(247, 8)

In [10]:
#P5
df_P5 = df.loc[df['period'] == 'P5']
df_P5.shape

(208, 8)

In [11]:
#P6
df_P6 = df.loc[df['period'] == 'P6']
df_P6.shape

(182, 8)

# Konkatenieren der einzelnen Teile

In [12]:
df_concat34 = pd.concat((df_P3, df_P4))
df_concat34.shape

(355, 8)

In [13]:
df_concat12 = pd.concat((df_P1, df_P2))
df_concat12.shape

(228, 8)

# Features P1 ist der Trainingskorpus, er heißt im folgenden _train

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words

tf = TfidfVectorizer(stop_words=get_stop_words('de'), max_features=20000)

#Wie beim Lennart steht das X für die Texte und Y für die Label

X_train = tf.fit_transform(df_P1.text)
X_P2 = tf.transform(df_P2.text)
X_P3 = tf.transform(df_P3.text)
X_P4 = tf.transform(df_P4.text)
X_P5 = tf.transform(df_P5.text)
X_P6 = tf.transform(df_P6.text)

y_train = df_P1.genre.to_numpy()
y_P2 = df_P2.genre.to_numpy()
y_P3 = df_P3.genre.to_numpy()
y_P4 = df_P4.genre.to_numpy()
y_P5 = df_P5.genre.to_numpy()
y_P6 = df_P6.genre.to_numpy()

# Naive Bayes P1 auf die restlichen einzelnen Perioden


In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

naive_bayes = MultinomialNB().fit(X_train, y_train)

y_pred_P2 = naive_bayes.predict(X_P2)
y_pred_P3 = naive_bayes.predict(X_P3)
y_pred_P4 = naive_bayes.predict(X_P4)
y_pred_P5 = naive_bayes.predict(X_P5)
y_pred_P6 = naive_bayes.predict(X_P6)
print('P2', classification_report(y_P2, y_pred_P2))
print('P3', classification_report(y_P3, y_pred_P3))
print('P4', classification_report(y_P4, y_pred_P4))
print('P5', classification_report(y_P5, y_pred_P5))
print('P6', classification_report(y_P6, y_pred_P6))

P2               precision    recall  f1-score   support

        DRAM       1.00      0.33      0.50        15
        HUMA       0.28      0.53      0.36        15
        LEGA       1.00      0.33      0.50        15
        NARR       0.54      0.93      0.68        15
        NEWS       1.00      1.00      1.00        23
        SCIE       0.78      0.47      0.58        15
        SERM       0.81      0.87      0.84        15

    accuracy                           0.66       113
   macro avg       0.77      0.64      0.64       113
weighted avg       0.79      0.66      0.66       113

P3               precision    recall  f1-score   support

        DRAM       1.00      0.13      0.24        15
        HUMA       0.35      0.80      0.49        15
        LEGA       1.00      0.27      0.42        15
        NARR       0.47      0.93      0.62        15
        NEWS       0.95      1.00      0.97        18
        SCIE       0.62      0.33      0.43        15
        SERM      

C:\Users\janko\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
print(y_pred_P6,'\n\n', y_P6) #Vergleich der vorhergesagten mit den tatsächlichen Label durch NB

['NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'SERM' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS

# Naive Bayes P1 und P2 concateniert auf die restlichen Perioden und den neuen Korpus

In [17]:
#Features:
X_train12 = tf.fit_transform(df_concat12.text)
X_P3 = tf.transform(df_P3.text)
X_P4 = tf.transform(df_P4.text)
X_P5 = tf.transform(df_P5.text)
X_P6 = tf.transform(df_P6.text)
X_inn = tf.transform(df_inn.text)

y_train12 = df_concat12.genre.to_numpy()
y_P3 = df_P3.genre.to_numpy()
y_P4 = df_P4.genre.to_numpy()
y_P5 = df_P5.genre.to_numpy()
y_P6 = df_P6.genre.to_numpy()
y_inn = df_inn.genre.to_numpy()

#Naive Bayes:
naive_bayes = MultinomialNB().fit(X_train12, y_train12)

y_pred_P3 = naive_bayes.predict(X_P3)
y_pred_P4 = naive_bayes.predict(X_P4)
y_pred_P5 = naive_bayes.predict(X_P5)
y_pred_P6 = naive_bayes.predict(X_P6)
y_pred_inn = naive_bayes.predict(X_inn)
print('P3', classification_report(y_P3, y_pred_P3))
print('P4', classification_report(y_P4, y_pred_P4))
print('P5', classification_report(y_P5, y_pred_P5))
print('P6', classification_report(y_P6, y_pred_P6))
print('Inn', classification_report(y_inn, y_pred_inn))

P3               precision    recall  f1-score   support

        DRAM       1.00      0.27      0.42        15
        HUMA       0.44      0.80      0.57        15
        LEGA       1.00      0.53      0.70        15
        NARR       0.47      1.00      0.64        15
        NEWS       1.00      1.00      1.00        18
        SCIE       1.00      0.33      0.50        15
        SERM       0.86      0.80      0.83        15

    accuracy                           0.69       108
   macro avg       0.82      0.68      0.66       108
weighted avg       0.83      0.69      0.67       108

P4               precision    recall  f1-score   support

        DRAM       1.00      0.41      0.58        22
        HUMA       0.00      0.00      0.00        24
        LEGA       0.00      0.00      0.00        25
        NARR       0.57      0.15      0.24        26
        NEWS       0.51      0.97      0.67       108
        SCIE       0.80      0.20      0.32        20
        SERM      

C:\Users\janko\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
print(y_pred_inn,'\n\n', y_inn)

['NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NARR' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'SERM' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS' 'SCIE' 'NEWS' 'NEWS' 'NEWS' 'SERM'
 'NEWS

# Naive Bayes P3 und P4 concateniert auf die restlichen Perioden

In [19]:
#Features:
X_train34 = tf.fit_transform(df_concat34.text)
X_P1 = tf.transform(df_P1.text)
X_P2 = tf.transform(df_P2.text)
X_P5 = tf.transform(df_P5.text)
X_P6 = tf.transform(df_P6.text)

y_train34 = df_concat34.genre.to_numpy()
y_P1 = df_P1.genre.to_numpy()
y_P2 = df_P2.genre.to_numpy()
y_P5 = df_P5.genre.to_numpy()
y_P6 = df_P6.genre.to_numpy()

#Naive Bayes:
naive_bayes = MultinomialNB().fit(X_train34, y_train34)

y_pred_P1 = naive_bayes.predict(X_P1)
y_pred_P2 = naive_bayes.predict(X_P2)
y_pred_P5 = naive_bayes.predict(X_P5)
y_pred_P6 = naive_bayes.predict(X_P6)
print('P1', classification_report(y_P1, y_pred_P1))
print('P2', classification_report(y_P2, y_pred_P2))
print('P5', classification_report(y_P5, y_pred_P5))
print('P6', classification_report(y_P6, y_pred_P6))

P1               precision    recall  f1-score   support

        DRAM       0.00      0.00      0.00        15
        HUMA       0.20      0.07      0.10        15
        LEGA       0.91      0.67      0.77        15
        NARR       0.38      0.87      0.53        15
        NEWS       1.00      1.00      1.00        25
        SCIE       0.00      0.00      0.00        15
        SERM       0.35      0.93      0.51        15

    accuracy                           0.55       115
   macro avg       0.41      0.50      0.42       115
weighted avg       0.46      0.55      0.47       115

P2               precision    recall  f1-score   support

        DRAM       0.00      0.00      0.00        15
        HUMA       0.33      0.13      0.19        15
        LEGA       0.91      0.67      0.77        15
        NARR       0.31      0.87      0.46        15
        NEWS       1.00      1.00      1.00        23
        SCIE       0.00      0.00      0.00        15
        SERM      

C:\Users\janko\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Naive Bayes alter auf neuer Korpus - Manchester Trainingskorpus

In [20]:
#Features:
X_train_man = tf.fit_transform(df_man.text)
X_test_inn = tf.transform(df_inn.text)

y_train_man = df_man.genre.to_numpy()
y_test_inn = df_inn.genre.to_numpy()

#Naive Bayes:
naive_bayes = MultinomialNB().fit(X_train_man, y_train_man)

y_pred_inn = naive_bayes.predict(X_test_inn)
print(classification_report(y_test_inn, y_pred_inn))

              precision    recall  f1-score   support

        DRAM       1.00      0.19      0.31        59
        HUMA       0.62      0.07      0.13        67
        LEGA       0.75      0.04      0.08        67
        NARR       0.65      0.16      0.26        67
        NEWS       0.43      0.97      0.60       245
     NEWS-P4       0.00      0.00      0.00         1
        SCIE       1.00      0.10      0.19        67
        SERM       0.87      0.51      0.64        65

    accuracy                           0.48       638
   macro avg       0.67      0.26      0.28       638
weighted avg       0.66      0.48      0.39       638



C:\Users\janko\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
print(y_pred_inn,'\n\n', y_test_inn)

['NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NARR' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'SERM' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS' 'SCIE' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS

# Naive Bayes neuer auf alter Korpus - Innsbruck Trainingskorpus

In [22]:
#Features:
X_train_inn = tf.fit_transform(df_inn.text)
X_test_man = tf.transform(df_man.text)

y_train_inn = df_inn.genre.to_numpy()
y_test_man = df_man.genre.to_numpy()

#Naive Bayes:
naive_bayes = MultinomialNB().fit(X_train_inn, y_train_inn)

y_pred_man = naive_bayes.predict(X_test_man)
print(classification_report(y_test_man, y_pred_man))

              precision    recall  f1-score   support

        DRAM       0.00      0.00      0.00        45
        HUMA       0.00      0.00      0.00        45
        LEGA       0.00      0.00      0.00        45
        NARR       0.00      0.00      0.00        45
        NEWS       0.21      1.00      0.35        66
        SCIE       0.00      0.00      0.00        45
        SERM       0.93      0.56      0.69        45

    accuracy                           0.27       336
   macro avg       0.16      0.22      0.15       336
weighted avg       0.17      0.27      0.16       336



C:\Users\janko\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
print(y_pred_inn,'\n\n', y_inn)

['NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NARR' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'SERM' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS' 'NEWS' 'SERM' 'NEWS' 'NEWS' 'SCIE' 'NEWS' 'NEWS' 'NEWS' 'NEWS'
 'NEWS

# Logistische Regression P1 auf den rest

In [24]:
X_train = tf.fit_transform(df_P1.text)
X_P2 = tf.transform(df_P2.text)
X_P3 = tf.transform(df_P3.text)
X_P4 = tf.transform(df_P4.text)
X_P5 = tf.transform(df_P5.text)
X_P6 = tf.transform(df_P6.text)

y_train = df_P1.genre.to_numpy()
y_P2 = df_P2.genre.to_numpy()
y_P3 = df_P3.genre.to_numpy()
y_P4 = df_P4.genre.to_numpy()
y_P5 = df_P5.genre.to_numpy()
y_P6 = df_P6.genre.to_numpy()

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

log_reg = LogisticRegression().fit(X_train, y_train)

y_pred_P2 = log_reg.predict(X_P2)
y_pred_P3 = log_reg.predict(X_P3)
y_pred_P4 = log_reg.predict(X_P4)
y_pred_P5 = log_reg.predict(X_P5)
y_pred_P6 = log_reg.predict(X_P6)
print(classification_report(y_P2, y_pred_P2))
print(classification_report(y_P3, y_pred_P3))
print(classification_report(y_P4, y_pred_P4))
print(classification_report(y_P5, y_pred_P5))
print(classification_report(y_P6, y_pred_P6))

              precision    recall  f1-score   support

        DRAM       1.00      0.33      0.50        15
        HUMA       0.26      0.73      0.39        15
        LEGA       1.00      0.33      0.50        15
        NARR       0.61      0.93      0.74        15
        NEWS       0.92      1.00      0.96        23
        SCIE       0.83      0.33      0.48        15
        SERM       1.00      0.47      0.64        15

    accuracy                           0.62       113
   macro avg       0.80      0.59      0.60       113
weighted avg       0.81      0.62      0.62       113

              precision    recall  f1-score   support

        DRAM       1.00      0.20      0.33        15
        HUMA       0.28      1.00      0.44        15
        LEGA       1.00      0.27      0.42        15
        NARR       0.56      0.67      0.61        15
        NEWS       0.72      1.00      0.84        18
        SCIE       0.75      0.20      0.32        15
        SERM       1.00 

C:\Users\janko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\janko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\janko\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Logistische Regression P1 und P2 konkateniert auf den Rest angewandt

In [26]:
#Features:
X_train12 = tf.fit_transform(df_concat12.text)
X_P3 = tf.transform(df_P3.text)
X_P4 = tf.transform(df_P4.text)
X_P5 = tf.transform(df_P5.text)
X_P6 = tf.transform(df_P6.text)
X_inn = tf.transform(df_inn.text)

y_train12 = df_concat12.genre.to_numpy()
y_P3 = df_P3.genre.to_numpy()
y_P4 = df_P4.genre.to_numpy()
y_P5 = df_P5.genre.to_numpy()
y_P6 = df_P6.genre.to_numpy()
y_inn = df_inn.genre.to_numpy()

#Logistische Regression:
log_reg = LogisticRegression().fit(X_train12, y_train12)

y_pred_P3 = log_reg.predict(X_P3)
y_pred_P4 = log_reg.predict(X_P4)
y_pred_P5 = log_reg.predict(X_P5)
y_pred_P6 = log_reg.predict(X_P6)
y_pred_inn = log_reg.predict(X_inn)
print('P3', classification_report(y_P3, y_pred_P3))
print('P4', classification_report(y_P4, y_pred_P4))
print('P5', classification_report(y_P5, y_pred_P5))
print('P6', classification_report(y_P6, y_pred_P6))
print('Inn', classification_report(y_inn, y_pred_inn))

C:\Users\janko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\janko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


P3               precision    recall  f1-score   support

        DRAM       1.00      0.27      0.42        15
        HUMA       0.40      0.93      0.56        15
        LEGA       1.00      0.47      0.64        15
        NARR       0.58      0.93      0.72        15
        NEWS       0.72      1.00      0.84        18
        SCIE       1.00      0.33      0.50        15
        SERM       1.00      0.53      0.70        15

    accuracy                           0.65       108
   macro avg       0.81      0.64      0.62       108
weighted avg       0.81      0.65      0.63       108

P4               precision    recall  f1-score   support

        DRAM       0.00      0.00      0.00        22
        HUMA       0.00      0.00      0.00        24
        LEGA       0.00      0.00      0.00        25
        NARR       0.00      0.00      0.00        26
        NEWS       0.44      1.00      0.61       108
        SCIE       0.00      0.00      0.00        20
        SERM      

C:\Users\janko\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Logistische Regression P3 und P4 angewandt auf den Rest

In [27]:
#Features:
X_train34 = tf.fit_transform(df_concat34.text)
X_P1 = tf.transform(df_P1.text)
X_P2 = tf.transform(df_P2.text)
X_P5 = tf.transform(df_P5.text)
X_P6 = tf.transform(df_P6.text)
X_inn = tf.transform(df_inn.text)

y_train34 = df_concat34.genre.to_numpy()
y_P1 = df_P1.genre.to_numpy()
y_P2 = df_P2.genre.to_numpy()
y_P5 = df_P5.genre.to_numpy()
y_P6 = df_P6.genre.to_numpy()
y_inn = df_inn.genre.to_numpy()

#Logistische Regression:
log_reg = LogisticRegression().fit(X_train34, y_train34)

y_pred_P1 = log_reg.predict(X_P1)
y_pred_P2 = log_reg.predict(X_P2)
y_pred_P5 = log_reg.predict(X_P5)
y_pred_P6 = log_reg.predict(X_P6)
y_pred_inn = log_reg.predict(X_inn)
print('P1', classification_report(y_P1, y_pred_P1))
print('P2', classification_report(y_P2, y_pred_P2))
print('P5', classification_report(y_P5, y_pred_P5))
print('P6', classification_report(y_P6, y_pred_P6))
print('Inn, (!!Vorsicht!! ist in P4 identisch mit trainingsdaten)', classification_report(y_inn, y_pred_inn))

C:\Users\janko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\janko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\janko\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


P1               precision    recall  f1-score   support

        DRAM       0.00      0.00      0.00        15
        HUMA       0.38      0.33      0.36        15
        LEGA       0.86      0.80      0.83        15
        NARR       0.42      0.67      0.51        15
        NEWS       1.00      1.00      1.00        25
        SCIE       0.75      0.60      0.67        15
        SERM       0.52      0.93      0.67        15

    accuracy                           0.65       115
   macro avg       0.56      0.62      0.58       115
weighted avg       0.60      0.65      0.61       115

P2               precision    recall  f1-score   support

        DRAM       0.00      0.00      0.00        15
        HUMA       0.39      0.47      0.42        15
        LEGA       0.87      0.87      0.87        15
        NARR       0.50      0.87      0.63        15
        NEWS       1.00      1.00      1.00        23
        SCIE       0.70      0.47      0.56        15
        SERM      

# Logistische Regression alter auf neuer - Manchester TRainingskorpus

In [28]:
#Features:
X_train_man = tf.fit_transform(df_man.text)
X_test_inn = tf.transform(df_inn.text)

y_train_man = df_man.genre.to_numpy()
y_test_inn = df_inn.genre.to_numpy()

#Logistische Regression:
log_reg = LogisticRegression().fit(X_train_man, y_train_man)

y_pred_inn = log_reg.predict(X_test_inn)
print(classification_report(y_test_inn, y_pred_inn))

C:\Users\janko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\janko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

        DRAM       0.00      0.00      0.00        59
        HUMA       0.00      0.00      0.00        67
        LEGA       0.00      0.00      0.00        67
        NARR       0.00      0.00      0.00        67
        NEWS       0.38      1.00      0.55       245
     NEWS-P4       0.00      0.00      0.00         1
        SCIE       0.00      0.00      0.00        67
        SERM       0.00      0.00      0.00        65

    accuracy                           0.38       638
   macro avg       0.05      0.12      0.07       638
weighted avg       0.15      0.38      0.21       638



C:\Users\janko\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Logistische Regression neuer auf alter Korpus - Innsbruck Trainingskorpus

In [29]:
#Features:
X_train_inn = tf.fit_transform(df_inn.text)
X_test_man = tf.transform(df_man.text)

y_train_inn = df_inn.genre.to_numpy()
y_test_man = df_man.genre.to_numpy()

#Logistische Regression:
log_reg = LogisticRegression().fit(X_train_inn, y_train_inn)

y_pred_man = log_reg.predict(X_test_man)
print(classification_report(y_test_man, y_pred_man))

C:\Users\janko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\janko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

        DRAM       1.00      0.20      0.33        45
        HUMA       1.00      0.07      0.12        45
        LEGA       0.00      0.00      0.00        45
        NARR       0.76      0.36      0.48        45
        NEWS       0.26      1.00      0.41        66
        SCIE       1.00      0.02      0.04        45
        SERM       0.84      0.82      0.83        45

    accuracy                           0.39       336
   macro avg       0.69      0.35      0.32       336
weighted avg       0.67      0.39      0.32       336



C:\Users\janko\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
